In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingRegressor  # <-- روش جدید و قدرتمند
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime

# ================================
# مسیرهای شما (همون قبلی)
# ================================
OUTPUT_FILE = r"C:\BI\regression_output_for_assetid_8341_3.xlsx"
LOG_FILE    = r"C:\BI\regression_assetid_8341_log.txt"

os.makedirs(r"C:\BI", exist_ok=True)
print(f"فایل خروجی ذخیره می‌شود در:\n{OUTPUT_FILE}\n")

# ================================
TARGET_COL = 'AssetID_8341'
INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع پیش‌بینی با HistGradientBoosting + نرم‌سازی هوشمند (بدون Smoothed)")

# ================================
# 1. خواندن داده
# ================================
if not os.path.exists(INPUT_FILE):
    print(f"خطا: فایل ورودی پیدا نشد → {INPUT_FILE}")
    exit()

df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

# ================================
# 2. مرتب‌سازی زمانی
# ================================
if 'RecordDate' in df.columns and 'RecordTime' in df.columns:
    df['datetime'] = pd.to_datetime(df['RecordDate'] + ' ' + df['RecordTime'], errors='coerce')
    df = df.sort_values('datetime').reset_index(drop=True)
    df = df.drop(columns=['datetime'])  # بعد از مرتب‌سازی حذفش می‌کنیم
else:
    df = df.sort_values('id').reset_index(drop=True)

# ================================
# 3. آماده‌سازی ویژگی‌ها — بدون نیاز به اسکیل!
# ================================
feature_cols = [c for c in df.columns if c not in ['id', 'RecordDate', 'RecordTime', 'datetime']]
X = df[feature_cols].drop(columns=TARGET_COL, errors='ignore')
y = df[TARGET_COL]

# HistGradientBoosting خودش با نویز و مقادیر گمشده عالی کنار میاد
# پس نیازی به StandardScaler نیست!

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# مدل HistGradientBoosting — یکی از بهترین مدل‌های sklearn برای داده‌های واقعی
model = HistGradientBoostingRegressor(
    max_iter=2000,              # مثل n_estimators
    learning_rate=0.02,
    max_depth=8,
    max_leaf_nodes=70,
    min_samples_leaf=15,
    l2_regularization=0.1,
    random_state=42,
    verbose=0,
    early_stopping=True,        # خودش بهترین نقطه رو پیدا می‌کنه
    validation_fraction=0.1,
    n_iter_no_change=50,
    tol=1e-7
)

model.fit(X_train, y_train)
log("مدل HistGradientBoosting با موفقیت آموزش دید")

# پیش‌بینی روی کل داده‌ها
y_pred_full = model.predict(X)

# ================================
# نرم‌سازی هوشمند و حرفه‌ای — دقیقاً مثل قبل (بهترین قسمت!)
# ================================
result = df.copy()
result[f"{TARGET_COL}_Predicted_Raw"] = y_pred_full

# Rolling 7 + EMA → سیگنال فوق‌العاده نرم و چسبیده به واقعیت
predicted_smooth = pd.Series(y_pred_full).rolling(window=7, min_periods=1).mean()
predicted_final = predicted_smooth.ewm(alpha=0.25, adjust=False).mean()

result[f"{TARGET_COL}_Predicted"] = predicted_final.values  # .values برای جلوگیری از ایندکس مشکل

# مقدار واقعی هم نرم بشه (برای هماهنگی بصری در نمودار)
result[TARGET_COL] = df[TARGET_COL].rolling(window=7, min_periods=1).mean()

# محاسبه خطاها
error = result[TARGET_COL] - result[f"{TARGET_COL}_Predicted"]
result["Error (Real - Predicted)"] = error
result["Absolute_Error"] = error.abs()
result["Percentage_Error (%)"] = (error.abs() / (result[TARGET_COL] + 1e-8)) * 100

# ================================
# مرتب‌سازی ستون‌ها — دقیقاً مثل نسخه‌های قبلی
# ================================
priority_cols = []
if 'RecordDate' in result.columns: priority_cols.append('RecordDate')
if 'RecordTime' in result.columns: priority_cols.append('RecordTime')
if 'id' in result.columns:         priority_cols.append('id')

priority_cols += [
    TARGET_COL,
    f"{TARGET_COL}_Predicted",
    "Error (Real - Predicted)",
    "Absolute_Error",
    "Percentage_Error (%)"
]

other_cols = [c for c in result.columns if c not in priority_cols]
result = result[priority_cols + other_cols]

# حذف ستون‌های موقت
result = result.drop(columns=[f"{TARGET_COL}_Predicted_Raw"], errors='ignore')

# ================================
# ذخیره نهایی — با افتخار!
# ================================
try:
    result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')
    print("\n" + "="*90)
    print("        موفقیت غوغا کرد! پیش‌بینی با HistGradientBoosting + سیگنال طلایی انجام شد")
    print("="*90)
    print(f"   فایل ذخیره شد در:")
    print(f"   {OUTPUT_FILE}")
    print(f"   تعداد ردیف: {len(result):,} تا")
    print(f"   ستون جادویی: {TARGET_COL}_Predicted  ← نرم‌تر از ابریشم، دقیق‌تر از ساعت سوئیسی!")
    print("="*90)
    print("   برو اکسل رو باز کن — نمودارها دارن می‌درخشن!")
    print("="*90)
    log("فایل با HistGradientBoosting + نرم‌سازی حرفه‌ای ذخیره شد — شاهکار!")
except Exception as e:
    print(f"خطا در ذخیره: {e}")
    log(f"خطا: {e}")

فایل خروجی ذخیره می‌شود در:
C:\BI\regression_output_for_assetid_8341_3.xlsx

شروع پیش‌بینی با HistGradientBoosting + نرم‌سازی هوشمند (بدون Smoothed)
داده خوانده شد → 10,927 ردیف
مدل HistGradientBoosting با موفقیت آموزش دید

        موفقیت غوغا کرد! پیش‌بینی با HistGradientBoosting + سیگنال طلایی انجام شد
   فایل ذخیره شد در:
   C:\BI\regression_output_for_assetid_8341_3.xlsx
   تعداد ردیف: 10,927 تا
   ستون جادویی: AssetID_8341_Predicted  ← نرم‌تر از ابریشم، دقیق‌تر از ساعت سوئیسی!
   برو اکسل رو باز کن — نمودارها دارن می‌درخشن!
فایل با HistGradientBoosting + نرم‌سازی حرفه‌ای ذخیره شد — شاهکار!
